In [1]:
#import libraries
import requests
import re
from bs4 import BeautifulSoup
import csv
import lxml


def get_html(url):                # performing the requests to the steam server
    response = requests.get(url)
    if not response.ok:  # status_code == 200:
        print(f'Code: {response.status_code}, url: {url}')
    return response.text


def write_csv(data):
    with open('result.csv', 'a') as f:                           # writing data to a csv file
        fields = ['title', 'released', 'reviews', 'tags']
        writer = csv.DictWriter(f, fieldnames=fields)
        writer.writerow(data)


def get_games(html):                                              # getting pure pages
    soup = BeautifulSoup(html, 'lxml')                              
    pattern = r'^https://store.steampowered.com/app/'
    games = soup.find_all('a', href=re.compile(pattern))
    return games


def get_hover_data(id):                                                               # getting desired data from pure page
    url = f'https://store.steampowered.com/apphoverpublic/{id}'                
    html = get_html(url)
    soup = BeautifulSoup(html, 'lxml')
    try:
        title = soup.find('h4', class_='hover_title').text.strip()
    except:
        title = ''
        print(url)
    try:
        released = soup.find('div', class_='hover_release').span.text.split(':')[-1].strip()
    except:
        released = ''
        print(url)
    try:
        reviews_raw = soup.find('div', class_='hover_review_summary').text
    except:
        reviews = ''
        print(url)
    else:
        pattern = r'\d+'
        reviews = int(''.join(re.findall(pattern, reviews_raw)))
    try:
        tags_raw = soup.find_all('div', class_='app_tag')
    except:
        tags = ''
        print(url)
    else:
        tags_text = [tag.text for tag in tags_raw]
        tags = ', '.join(tags_text)

    data = {                                                                                          # joining all gathered data together
        'title': title,
        'released': released,
        'reviews': reviews,
        'tags': tags
    }

    write_csv(data)


def main():
    all_games = []
    start = 0
    url = f'https://store.steampowered.com/search/results/?query&start={start}&count=100&tags=1702'

    while True:

        games = get_games(get_html(url))

        if games:
            all_games.extend(games)
            start += 100
            url = f'https://store.steampowered.com/search/results/?query&start={start}&count=100&tags=1702'         # we're counting from 0 to 100
        else:
            break

    for game in all_games:
        id = game.get('data-ds-appid')
        get_hover_data(id)


if __name__ == '__main__':
    main()


https://store.steampowered.com/apphoverpublic/219740
https://store.steampowered.com/apphoverpublic/729000
https://store.steampowered.com/apphoverpublic/612660
https://store.steampowered.com/apphoverpublic/1312960
https://store.steampowered.com/apphoverpublic/759300
https://store.steampowered.com/apphoverpublic/1298550
https://store.steampowered.com/apphoverpublic/1323280
https://store.steampowered.com/apphoverpublic/1044980
https://store.steampowered.com/apphoverpublic/1051440
https://store.steampowered.com/apphoverpublic/1315020
https://store.steampowered.com/apphoverpublic/1285690
https://store.steampowered.com/apphoverpublic/1248800
https://store.steampowered.com/apphoverpublic/504030
https://store.steampowered.com/apphoverpublic/1125390
https://store.steampowered.com/apphoverpublic/454120
https://store.steampowered.com/apphoverpublic/1324910
https://store.steampowered.com/apphoverpublic/1059900
https://store.steampowered.com/apphoverpublic/1320100
https://store.steampowered.com/app

Scrape data that loaded by ajax requests and one of the solutions to do it is to use a selenium webdriver but the selenium is heavy thin and the scrape with it is a rather slow process so I would scrape ajax website with the only requests library and beautifulsoup library. I used the steam store as an example.

This page is an infinite page it means that when I'm scrolling down the content the content of the next pages is loaded automatically by some JavaScript code and also when I'm hovering a mouse cursor over a game item in the list the small window is showing up on the right side and this window is also loaded by some JavaScript and there the title of the game, it's release date, total users review and user text. Exactly this date I want to scrape and save to a csv file.